In [179]:
import pandas as pd

### 1
Давайте проанализируем данные опроса 4361 женщин из Ботсваны:  
<br>  
<center>botswana.tsv </center>
   
    
О каждой из них мы знаем:
- сколько детей она родила (признак ceb)
- возраст (age)
- длительность получения образования (educ)
- религиозная принадлежность (religion)
- идеальное, по её мнению, количество детей в семье (idlnchld)
- была ли она когда-нибудь замужем (evermarr)
- возраст первого замужества (agefm)
- длительность получения образования мужем (heduc)
- знает ли она о методах контрацепции (knowmeth)
- использует ли она методы контрацепции (usemeth)
- живёт ли она в городе (urban)
- есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)
- Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [180]:
df = pd.read_csv("botswana.tsv", sep="\t")

In [181]:
df.religion.unique().shape[0]

4

---
### 2
Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

In [182]:
df.dropna().shape[0]

1834

---
### 3

В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке **agefm** пропуски стоят только там, где **evermarr=0**, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака **NaN** соответствует значению "не применимо".

В подобных случаях, когда признак $x_1$ на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

- создать новый бинарный признак 

\begin{equation}
x_2 =
\begin{cases}
1, & x_1=\text{ 'не применимо'}, \\
0, & \text{иначе};
\end{cases}
\end{equation}


- заменить "не применимо" в $x_1$ на произвольную константу $c$, которая среди других значений $x_1$ не встречается.


Теперь, когда мы построим регрессию на оба признака и получим модель вида

$$ y=\beta_0 + \beta_1 x_1 + \beta_2 x_2,$$ 

на тех объектах, где $x_1$ было измерено, регрессионное уравнение примет вид

$$y=\beta_0 + \beta_1 x,$$ 
  
было "не применимо", получится

$$ y=\beta_0 + \beta_1 c + \beta_2.$$


Выбор $c$ влияет только на значение и интерпретацию $\beta_2$, но не $\beta_1$.

---

Давайте используем этот метод для обработки пропусков в **agefm** и **heduc**.


1. Создайте признак **nevermarr**, равный единице там, где в **agefm** пропуски.


2. Удалите признак **evermarr** — в сумме с **nevermarr** он даёт константу, значит, в нашей матрице $X$ будет мультиколлинеарность.


3. Замените *NaN* в признаке **agefm** на $c_{agefm}=0$. 


4. У объектов, где **nevermarr** = 1, замените NaN в признаке **heduc** на $c_{heduc\_1}=-1$ (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).


Сколько осталось пропущенных значений в признаке heduc?

In [183]:
# Создайте признак nevermarr, равный единице там, где в agefm пропуски

df["nevermarr"] = 0
df.loc[df.agefm.isna(), "nevermarr"] = 1  # можно отобразить evermarr

In [184]:
# Удалите признак evermarr 
df.drop("evermarr", axis=1, inplace=True)

In [185]:
# Замените NaN в признаке agefm на 0
df.agefm.fillna(0, inplace=True)

In [186]:
# У объектов, где nevermarr = 1, замените NaN в признаке heduc на -1
df.loc[df.nevermarr == 1, "heduc"] = -1

In [187]:
# Сколько осталось пропущенных значений в признаке heduc
df.heduc.isna().value_counts()

False    4238
True      123
Name: heduc, dtype: int64

---
### 4
Избавимся от оставшихся пропусков.

Для признаков **idlnchld**, **heduc** и **usemeth** проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения 
- $c_{idlnchld}=-1$,  
- $c_{heduc\_2}=-2$ (значение -1 мы уже использовали), 
- $c_{usemeth}=-1$.

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [188]:
df["idlnchld_noans"] = 0
df.loc[df.idlnchld.isna(), "idlnchld_noans"] = 1

df["heduc_noans"] = 0
df.loc[df.heduc.isna(), "heduc_noans"] = 1

df["usemeth_noans"] = 0
df.loc[df.usemeth.isna(), "usemeth_noans"] = 1

In [189]:
for name, value in zip(["idlnchld", "heduc", "usemeth"], (-1, -2, -1)):
    df[name].fillna(value, inplace=True)

In [190]:
df.dropna(inplace=True)

In [191]:
df.shape[0] * df.shape[1]

78264

---
### 5
Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации $R^2$? Округлите до трёх знаков после десятичной точки.

In [192]:
df.head(1)

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0


In [193]:
import statsmodels.formula.api as smf

In [194]:
formula = "ceb ~ " + " + ".join(df.columns[1:])
res = smf.ols(formula=formula, data=df)
fitted = res.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Thu, 07 Nov 2019   Prob (F-statistic):               0.00
Time:                        18:38:59   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [195]:
0.644

0.644

---
### 6
Обратите внимание, что для признака *religion* в модели автоматически создалось несколько бинарных фиктивных переменных. Сколько их?

In [196]:
3

3

---
### 7
Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Если ошибка гетероскедастична, перенастройте модель, сделав поправку Уайта типа HC1.


- Ошибка гомоскедастична, $p>0.05$


- Ошибка гетероскедастична, $p\leq0.05$, нужно делать поправку Уайта $\checkmark$



In [197]:
import statsmodels.stats.api as sms

In [198]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


---
### 8
Удалите из модели незначимые признаки **religion**, **radio** и **tv**. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [199]:
df2 = df.drop(["religion", "radio", "tv"], axis=1)

In [200]:
formula_2 = "ceb ~ " + " + ".join(df2.columns[1:])

In [201]:
fitted_1 = smf.ols(formula=formula, data=df).fit(cov_type='HC1')
fitted_2 = smf.ols(formula=formula_2, data=df).fit(cov_type='HC1')

In [203]:
res = fitted.compare_f_test(fitted_2)
print("F=%f, p=%f, k1=%f" % res, "; deleted 3 features")
print(round(res[1], 4))

F=0.919236, p=0.467231, k1=5.000000 ; deleted 3 features
0.4672


D:\Anaconda3\envs\py36_R\lib\site-packages\statsmodels\regression\linear_model.py:2018: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)


---
### 9
Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением $c_{usemeth }=-1$, удалять *usemeth_noans* и _usemeth_ можно только вместе.

Удалите из текущей модели *usemeth_noans* и _usemeth_. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили $ 5.5\times10^{-8}$, нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без _usemeth_ и *usemeth_noans*.

In [204]:
cols_3 = ['age', 'educ', 'idlnchld', 'knowmeth', 'agefm',
       'heduc', 'urban', 'electric', 'bicycle', 'nevermarr', 'idlnchld_noans',
       'heduc_noans']
formula_3 = "ceb ~ " + " + ".join(cols_3)
formula_3

'ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans'

In [205]:
fitted_3 = smf.ols(formula=formula_3, data=df).fit(cov_type='HC1')

In [206]:
fitted_2.compare_f_test(fitted_3)

D:\Anaconda3\envs\py36_R\lib\site-packages\statsmodels\regression\linear_model.py:2018: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)


(92.89058230109713, 3.155200948040263e-40, 2.0)

In [207]:
40

40

In [208]:
print(fitted_2.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Thu, 07 Nov 2019   Prob (F-statistic):               0.00
Time:                        18:39:01   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

---
### 10
Посмотрите на доверительные интервалы для коэффициентов итоговой модели (не забудьте использовать поправку Уайта, если есть гетероскедастичность ошибки) и выберите правильные выводы.


- У женщин, знакомых с методами контрацепции **knowmeth**, при прочих равных в среднем на 0.6 ребёнка меньше (p=0.001, 95% доверительный интервал для разницы между средними — [-0.9, -0.2]) **(нет, так как коэф положительный)**


- У женщин, не знающих, какое количество детей идеально (**idlnchld_noans**), в среднем на 0.66 ребёнка больше (p=0.002, 95% доверительный интервал — [0.2, 1.1])


-  женщин, никогда не выходивших замуж (**nevermarr**), при прочих равных в среднем на 2.3 ребёнка меньше (p<0.001, 95% доверительный интервал для разницы между средними — [-2.6, -1.9]) $\large\checkmark$ 


- С увеличением возраста женщины на 1 год среднее количество детей возрастает на 0.17 (p<0.001, 95% доверительный интервал — [0.16, 0.18]) $\large\checkmark$


- У женщин, не знающих, какое количество детей идеально (**idlnchld_noans**), в среднем на 
$$ β_{idlnchld\_noans}+c_{idlnchld}β_{idlnchld} ≈ 0.58 $$ детей больше *$\large\checkmark$